In [1]:
%cd ..

C:\Users\kubak\Desktop\engineer-project\github\LanguageModule


C:\Users\kubak\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import cohere
import os
import json

# Łączenie się z api cohere

In [3]:
# Inicjalizacja klienta Cohere
# https://cohere.com/
# https://docs.cohere.com/reference/embed

with open('./api.txt', 'r', encoding='utf-8') as file:
    for line in file:
        if 'cohere:' in line:
            # Wyciągamy tekst po "cohere:"
            key_api = line.split('cohere:')[1].strip()
            
co = cohere.Client(key_api)

# Pobieranie dokumentów

In [4]:
documents_path =  f"./data/decoded_pdfs/"

In [5]:
def split_into_paragraphs(text):
    # Split the text into paragraphs by empty lines
    paragraphs = [paragraph.strip() for paragraph in text.split('\n\n') if paragraph.strip()]
    return paragraphs

In [6]:
# Lista do przechowywania zawartości książek
documents = []

# Iteracja przez wszystkie pliki w folderze
for i, filename in enumerate(os.listdir(documents_path)):
    if filename.endswith(".json"):  # Sprawdzenie, czy plik ma rozszerzenie .json
        file_path = os.path.join(documents_path, filename)
        
        # Otwórz plik JSON i załaduj dane
        with open(file_path, "r", encoding="utf-8") as file:
            book_content = json.load(file)

        # Dodaj każdą stronę oddzielnie
        for page in book_content.values():
            # Zamieniamy stronę na akapity, w domyśle zakładamy że najmniejszą jednostką znaczeniową jest akapit.
            paragraphs = split_into_paragraphs(page)

            for j, paragraph in enumerate(paragraphs):
                documents.append(paragraph)

        # Połącz zawartość wszystkich stron w jedną zmienną tekstową
        #full_text = " ".join(book_content.values())
        # Dodaj pełny tekst książki do listy documents
        #documents.append(full_text)

In [19]:
print(len(documents))

2591


# RAG - łączenie dokumentów

In [8]:
# Funkcja do wyszukiwania dokumentów w zbiorze
def retrieve_relevant_documents(query, documents):
    response = co.rerank(
        model='rerank-english-v3.0',
        query=query,
        documents=documents,
        top_n=5  # Liczba dokumentów do zwrócenia
    )
    return response

# Funkcja do generowania odpowiedzi
def generate_response(query, prompt):
    response = co.generate(
        model="command",  #-xlarge-nightly",  # Model generowania Cohere
        prompt=prompt,
        max_tokens=100,  # Maksymalna długość odpowiedzi
        temperature=0.7  # Stopień kreatywności odpowiedzi
    )
    return response.generations[0].text.strip()


In [25]:
# Bierzemy dokumenty z minimalną trafnością:
minimum_relevance = 0.5

# Bierzemy maksymalnie w rachubę:
maximum_documents = 3

# Przykład zapytania użytkownika
query = "What is a low bar squat?"

In [26]:
# Analiza wyników z indexami
def analyze_rerank_results(response, documents):
    results = response.results  # Lista wyników
    relevant_docs = []

    for result in results:
        index = result.index  # Indeks w liście `documents`
        relevance_score = result.relevance_score  # Ocena trafności

        # Pobranie dokumentu na podstawie indeksu
        document_content = documents[index] if index is not None and index < len(documents) else None

        relevant_docs.append({
            "index": index,
            "relevance_score": relevance_score,
            "document": document_content  # Treść dokumentu
        })

    return relevant_docs

In [27]:
# Krok 1: Znalezienie najbardziej adekwatnego dokumentu
retrieval_result = retrieve_relevant_documents(query, documents)


In [28]:
relevant_docs = analyze_rerank_results(retrieval_result, documents)

# Wyświetlenie wyników
for doc in relevant_docs:
    print(f"Indeks: {doc['index']}, Trafność: {doc['relevance_score']}: \n Dokument: {doc['document']} \n")

Indeks: 1812, Trafność: 0.99657035: 
 Dokument: low-bar position, with the weight sitting just below  the  spine of the scapula,  is  the  same as the mechanics of 
the  position in which the bar is pulled off of the floor.  As  the  discussions of pulling mechanics in  the 
deadlift and power clean chapters have  illustrated,  the  shoulder blades  are  directly above  the bar when  it 
leaves the floor, and they stay there until  the  back changes position when the bar rises above the knees. 
(This is true for both the clean and the snatch,  with  the  snatch being done  from  a position even less similar to 
the  Olympic squat than the clean.)  Low-bar squats done utilizing this same relative position will train  the 
movement partem more directly  than  the  high-bar version, which places the back at  a higher angle due  to the 
higher position of the bar on  the  traps,  and  the  scapulas  behind the position  they  are  in  during  a  pull.  And 
they do it through a nice, long 

In [31]:
# Bierzemy maksymalnie w rachubę:
maximum_documents = 3
filtered_docs = [doc for doc in relevant_docs if doc["relevance_score"] > minimum_relevance]

if(len(filtered_docs) >= maximum_documents):
    filtered_docs = filtered_docs[:maximum_documents]

In [37]:
# Krok 2: Wygenerowanie odpowiedzi lub fallback
if len(filtered_docs) > 0:

    # Połaczenie wszystkich wyników
    context = " ".join([doc["document"] for doc in filtered_docs])

    # Tworzenie promptu z zapytania i znalezionego kontekstu
    prompt = f"User question: {query}\n Context: {context}\n Answer:"

    response = generate_response(query, most_relevant_document)
else:
    response = "Przepraszam, nie znalazłem odpowiedniego dokumentu, ale mogę spróbować odpowiedzieć ogólnie."

# Wyświetlenie wyniku
print("Pytanie użytkownika:", query)
print("Odpowiedź:", response)

Pytanie użytkownika: What is a low bar squat?
Odpowiedź: A low bar squat is a variation of the squat exercise, one of the most fundamental strength training exercises. 

Unlike a high bar squat, where the bar rests on the trapezius muscles, in a low bar squat, the bar is positioned lower down the back, often resting slightly above the rear deltoids. 

Low bar squats tend to emphasize the quadriceps muscles to a greater degree than high bar squats, and are often favored by strength athletes such as powerlif


# Embeding model

In [42]:
# Krojenie dokumentów
documents_shortend = documents[1:100]

# Tworzenie embeddingów dokumentów
embeddings = co.embed(
    model="embed-english-v3.0",
    texts=documents_shortend,
    input_type="search_document"  
).embeddings

In [43]:
import numpy as np

# Funkcja obliczania podobieństwa kosinusowego
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Wyszukiwanie najbliższego dokumentu
def search(query, documents, embeddings):
    # Generowanie embeddingu dla zapytania
    query_embedding = co.embed(
        texts=[query],
        model="embed-english-v3.0",
        input_type="search_query"
    ).embeddings[0]
    
    # Obliczanie podobieństw
    similarities = [cosine_similarity(query_embedding, doc_emb) for doc_emb in embeddings]
    best_match_index = np.argmax(similarities)
    return documents[best_match_index], similarities[best_match_index]

In [46]:
def generate_answer(query):
    # Wyszukiwanie kontekstu
    context, similarity = search(query, documents, embeddings)

    print("context", context)
    # Generowanie odpowiedzi na podstawie kontekstu
    prompt = f"Question: {query}\n Context: {context}\n Answer:"
    response = co.generate(
        model="command-xlarge-nightly",
        prompt=prompt,
        max_tokens=200,
        temperature=0.7
    )
    return response.generations[0].text.strip()



In [47]:
# Testowanie
query = "What is low bar squat?"
answer = generate_answer(query)
print("Odpowiedź:", answer)

context Learning  to  squat
Odpowiedź: The low-bar squat is a variant of the back squat where the bar is held lower down on the back. This technique places greater emphasis on the hip muscles, such as the adductors and glutes, and reduces the involvement of the quadriceps.
